In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3fe3fdcc-f90e-4aa0-8281-4b06c30fb2a0;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from datetime import datetime
from typing import List
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from delta.tables import DeltaTable

In [5]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:
            schema = StructType() \
                    .add("key",BinaryType(),True) \
                    .add("value",BinaryType(),True) \
                    .add("topic",StringType(),True) \
                    .add("partition",IntegerType(),True) \
                    .add("offset",StringType(),True) \
                    .add("timestamp",TimestampType(),True)
            df = (
                spark
                .read
                .format(source_format)
                .schema(schema)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df


def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        df_r = df.where((f.col('__op') == 'r'))
        (
            df_r
            .write
            .format(destination_format)
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

In [6]:

#PATH SILVER ZONE
SILVER_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda/'
SILVER_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
SILVER_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial/'
SILVER_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
SILVER_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
SILVER_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
SILVER_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
SILVER_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
SILVER_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/site/'
SILVER_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
SILVER_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

SILVER_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos/'
SILVER_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
SILVER_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
SILVER_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
SILVER_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

SILVER_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'



#PATH GOLD ZONE
GOLD_PATH_CUSTOMERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/customers'
GOLD_PATH_ORDERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders/'
GOLD_PATH_ORDERS_ITEMS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders_items/'
GOLD_PATH_PRODUCTS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/products/'
GOLD_PATH_SELLERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/sellers/'
GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/stores/'

GOLD_PATH_OTO_DZARM = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/dzarm/'
GOLD_PATH_OTO_HERING_HMG = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_hmg/'
GOLD_PATH_OTO_HERING_KIDS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_kids/'
GOLD_PATH_OTO_HERING_OUTLETS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_outlets/'

GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/stores'

In [14]:
produto_barra  = DeltaTable.forPath(spark, SILVER_PATH_PRODUTOS_BARRA_LINX)
produtos  = DeltaTable.forPath(spark, SILVER_PATH_PRODUTOS)
produto_cores = DeltaTable.forPath(spark, SILVER_PATH_PRODUTO_CORES)
catalogo_sku = DeltaTable.forPath(spark, SILVER_PATH_HR_WEB_CATALOGO_SKU)
catalogo_foto = DeltaTable.forPath(spark, SILVER_PATH_HR_WEB_CATALOGO_FOTO)

stores = DeltaTable.forPath(spark, GOLD_PATH_STORES)

In [72]:
produto_barra.toDF().createOrReplaceTempView("view_produto_barra_linx")
produtos.toDF().createOrReplaceTempView("view_produto")
produto_cores.toDF().createOrReplaceTempView("view_produto_cores")
catalogo_sku.toDF().createOrReplaceTempView("view_hr_web_catalogo_sku")
catalogo_foto.toDF().createOrReplaceTempView("view_hr_web_catalogo_foto")


stores.toDF().createOrReplaceTempView("view_stores")

In [70]:
catalogo_foto.toDF().limit(5).toPandas()

__op transaction         timestamp_kafka CODIGO_SITE COR_PRODUTO  \
0    r        None 2022-05-25 19:43:44.563         001       XLVEN   
1    r        None 2022-05-25 19:43:44.563         001       XLVEN   
2    r        None 2022-05-25 19:43:44.563         001       XLVEN   
3    r        None 2022-05-25 19:43:44.563         001       XLVEN   
4    r        None 2022-05-25 19:43:44.563         001       XLVEN   

          DATA_IMPORTACAO  EXCLUIDO  FOTO_LOCAL                 HASH_FOTO  \
0 2022-01-06 15:01:06.957      True       False  G8Dq6DUAKlqHbc35J1d9TQ==   
1 2022-01-08 15:01:39.403      True       False  I66IvytmF4lq3PGx4BNwUg==   
2 2021-12-04 15:01:26.220      True       False  LONWPIanc+K2pYWbEDD5+w==   
3 2022-01-04 15:01:13.593      True       False  OcVLKBv3cOAxsiSrC4N69w==   
4 2022-02-25 15:01:33.393      True       False  Bq0I4Cs97dAZFqTfn4bbGA==   

                                           LINK_FOTO NOME_LOCAL PRODUTO  \
0  https://static.hering.com.br//sys_master/image...       None    0227   
1  https://static.hering.com.br//sys_master/image...       None    0227   
2  https://static.hering.com.br//sys_master/image...       None    0227   
3  https://static.hering.com.br//sys_master/image...       None    0227   
4  https://static.hering.com.br//sys_master/image...       None    0227   

      pk_single  
0  0010227XLVEN  
1  0010227XLVEN  
2  0010227XLVEN  
3  0010227XLVEN  
4  0010227XLVEN

In [62]:
produtos.toDF().select('PRODUTO').limit(5).toPandas()

PRODUTO
0  012N        
1  014V        
2  015W        
3  01YP        
4  0295

In [85]:
spark.sql('''
SELECT PRODUTO.PRODUTO,

(SELECT LINK_FOTO FROM view_hr_web_catalogo_foto AS HR_WEB_CATALOGO_FOTO 
    WHERE trim(HR_WEB_CATALOGO_FOTO.PRODUTO) = trim(PRODUTO.PRODUTO) limit 1
) AS TESTE    

FROM view_produto as PRODUTO WHERE  trim(PRODUTO.PRODUTO) = '012N'
         
''').limit(5).toPandas()

AnalysisException: Can't extract value from PRODUTO#3596: need struct type but got string; line 5 pos 52

In [74]:
spark.sql('''
SELECT PRODUTO.PRODUTO
FROM view_produto as PRODUTO
         
''').limit(5).toPandas()

PRODUTO
0  012N        
1  014V        
2  015W        
3  01YP        
4  0295

In [105]:
result_sql = spark.sql( '''

   
-----------------------------HERING STORE--------------------
(SELECT
    PRODUTOS_BARRA.SKU AS PRODUCT_SKU
    ,PRODUTOS.PRODUTO AS PRODUCT_ID
    ,IFNULL(PRODUTOS.TITULO_B2C,PRODUTOS.DESC_PRODUTO) AS  PRODUCT_NAME
    ,PRODUTOS_BARRA.GRADE AS PRODUCT_SIZE
    ,PRODUTOS.GRUPO_PRODUTO AS PRODUCT_CATEGORY
    ,PRODUTOS.SUBGRUPO_PRODUTO AS PRODUCT_SUBCATEGORY
    ,CASE WHEN PRODUTOS.SEXO_TIPO = 2 THEN 'MASCULINO'
         WHEN PRODUTOS.SEXO_TIPO = 3 THEN 'FEMININO'
         WHEN PRODUTOS.SEXO_TIPO = 4 THEN 'UNISSEX'
         ELSE 'INDEFINIDO'
    END   AS PRODUCT_GENDER
    ,PRODUTOS.LINHA AS PRODUCT_LINE
    ,PRODUTO_CORES.DESC_COR_PRODUTO AS PRODUCT_COLOR
    ,PRODUTOS.GRIFFE AS PRODUCT_BRAND
    ,first((SELECT first(LINK_FOTO) FROM  view_hr_web_catalogo_foto AS HR_WEB_CATALOGO_FOTO
        WHERE HR_WEB_CATALOGO_FOTO.PRODUTO = PRODUTO_CORES.PRODUTO
            AND HR_WEB_CATALOGO_FOTO.COR_PRODUTO = PRODUTO_CORES.COR_PRODUTO
            AND EXCLUIDO =0 AND HR_WEB_CATALOGO_FOTO.CODIGO_SITE='001'
         )) AS PRODUCT_IMAGE
    ,MAX(HR_WEB_CATALOGO_SKU.LINK) AS PRODUCT_URL
    ,PRODUTOS.__op
    ,PRODUTOS.timestamp_kafka
    ,'HERING STORE' AS MARCA
FROM view_produto_barra_linx AS PRODUTOS_BARRA
INNER JOIN view_produto AS PRODUTOS
    ON PRODUTOS_BARRA.PRODUTO = PRODUTOS.PRODUTO
INNER JOIN view_produto_cores AS PRODUTO_CORES
    ON PRODUTOS_BARRA.PRODUTO=PRODUTO_CORES.PRODUTO
    AND PRODUTOS_BARRA.COR_PRODUTO=PRODUTO_CORES.COR_PRODUTO
LEFT JOIN  view_hr_web_catalogo_sku AS HR_WEB_CATALOGO_SKU
    ON PRODUTOS_BARRA.PRODUTO=HR_WEB_CATALOGO_SKU.PRODUTO
    AND PRODUTOS_BARRA.COR_PRODUTO=HR_WEB_CATALOGO_SKU.COR_PRODUTO
    AND HR_WEB_CATALOGO_SKU.TAMANHO =PRODUTOS_BARRA.GRADE
    AND HR_WEB_CATALOGO_SKU.CODIGO_SITE='001' /*(CODIGO SITE HERING)*/
WHERE PRODUTOS.GRIFFE LIKE 'HERING%' /*(SO MARCAS HERING)*/
    AND PRODUTO_CORES.HR_INATIVO =0
    AND PRODUTOS.INATIVO =0
GROUP BY PRODUTOS_BARRA.SKU,
PRODUTOS.PRODUTO,
PRODUTO_CORES.PRODUTO,
PRODUTO_CORES.COR_PRODUTO,
PRODUTOS.SEXO_TIPO,
IFNULL(PRODUTOS.TITULO_B2C,PRODUTOS.DESC_PRODUTO) ,
PRODUTOS_BARRA.GRADE,
PRODUTOS.GRUPO_PRODUTO,
PRODUTOS.SUBGRUPO_PRODUTO,
PRODUTOS.LINHA,
PRODUTO_CORES.DESC_COR_PRODUTO,
PRODUTOS.GRIFFE,
PRODUTOS.__op,
PRODUTOS.timestamp_kafka)

'''
                      )

In [106]:
result_sql = spark.sql(sql)

ParseException: 
mismatched input 'first' expecting {')', ',', '-'}(line 20, pos 18)

== SQL ==


   
-----------------------------HERING STORE--------------------
(SELECT
    PRODUTOS_BARRA.SKU AS PRODUCT_SKU
    ,PRODUTOS.PRODUTO AS PRODUCT_ID
    ,IFNULL(PRODUTOS.TITULO_B2C,PRODUTOS.DESC_PRODUTO) AS  PRODUCT_NAME
    ,PRODUTOS_BARRA.GRADE AS PRODUCT_SIZE
    ,PRODUTOS.GRUPO_PRODUTO AS PRODUCT_CATEGORY
    ,PRODUTOS.SUBGRUPO_PRODUTO AS PRODUCT_SUBCATEGORY
    ,CASE WHEN PRODUTOS.SEXO_TIPO = 2 THEN 'MASCULINO'
         WHEN PRODUTOS.SEXO_TIPO = 3 THEN 'FEMININO'
         WHEN PRODUTOS.SEXO_TIPO = 4 THEN 'UNISSEX'
         ELSE 'INDEFINIDO'
    END   AS PRODUCT_GENDER
    ,PRODUTOS.LINHA AS PRODUCT_LINE
    ,PRODUTO_CORES.DESC_COR_PRODUTO AS PRODUCT_COLOR
    ,PRODUTOS.GRIFFE AS PRODUCT_BRAND
    ,first(SELECT first(LINK_FOTO) FROM  view_hr_web_catalogo_foto AS HR_WEB_CATALOGO_FOTO
------------------^^^
        WHERE HR_WEB_CATALOGO_FOTO.PRODUTO = PRODUTO_CORES.PRODUTO
            AND 

In [107]:
result_sql.limit(5).toPandas()

PRODUCT_SKU    PRODUCT_ID                                 PRODUCT_NAME  \
0  0111A1N07S6  0111          Regata Básica Masculina Em Malha De Algodão   
1  0111A3907S4  0111          Regata Básica Masculina Em Malha De Algodão   
2   0111AN6EN2  0111          Regata Básica Masculina Em Malha De Algodão   
3  0111AP907S4  0111          Regata Básica Masculina Em Malha De Algodão   
4  0111N0A00S3  0111          Regata Básica Masculina Em Malha De Algodão   

  PRODUCT_SIZE PRODUCT_CATEGORY PRODUCT_SUBCATEGORY PRODUCT_GENDER  \
0     XXG               REGATAS               ALCAS      MASCULINO   
1     G                 REGATAS               ALCAS      MASCULINO   
2          P            REGATAS               ALCAS      MASCULINO   
3          G            REGATAS               ALCAS      MASCULINO   
4     M                 REGATAS               ALCAS      MASCULINO   

  PRODUCT_LINE PRODUCT_COLOR PRODUCT_BRAND PRODUCT_IMAGE PRODUCT_URL __op  \
0       BASICO          AZUL        HERING          None        None    u   
1       BASICO          AZUL        HERING          None        None    u   
2       BASICO          AZUL        HERING          None        None    u   
3       BASICO          AZUL        HERING          None        None    u   
4       BASICO        BRANCO        HERING          None        None    u   

          timestamp_kafka         MARCA  
0 2022-05-26 04:44:32.352  HERING STORE  
1 2022-05-26 04:44:32.352  HERING STORE  
2 2022-05-26 04:44:32.352  HERING STORE  
3 2022-05-26 04:44:32.352  HERING STORE  
4 2022-05-26 04:44:32.352  HERING STORE

In [31]:
    her_store = result_sql.where(result_sql.MARCA == 'HERING STORE')
    her_kids = result_sql.where(result_sql.MARCA == 'HERING KIDS')
    her_outlet = result_sql.where(result_sql.MARCA == 'HERING OUTLET')
    dzarm = result_sql.where(result_sql.MARCA == 'DZARM')

In [32]:
    try:
        delta_her_store = DeltaTable.forPath(spark, GOLD_PATH_OTO_HERING_HMG + 'products/')
        delta_her_kids = DeltaTable.forPath(spark, GOLD_PATH_OTO_HERING_KIDS + 'products/')
        delta_her_outlet = DeltaTable.forPath(spark, GOLD_PATH_OTO_HERING_OUTLETS + 'products/')
        delta_dzarm = DeltaTable.forPath(spark, GOLD_PATH_OTO_DZARM + 'products/')
        is_incremental = True
    except:
        is_incremental = False

In [108]:
is_incremental

False

In [33]:
    if is_incremental:
        d = datetime.today() - timedelta(days=3)

        her_store = her_store.where(result_sql.timestamp_kafka > d)
        her_kids = her_kids.where(result_sql.timestamp_kafka > d)
        her_outlet = her_outlet.where(result_sql.timestamp_kafka > d)
        dzarm = dzarm.where(result_sql.timestamp_kafka > d)
        
        update_new_data(spark, her_store, GOLD_PATH_OTO_HERING_HMG + 'products/', ['PRODUCT_ID'])
        update_new_data(spark, her_kids, GOLD_PATH_OTO_HERING_KIDS + 'products/', ['PRODUCT_ID'])
        update_new_data(spark, her_outlet, GOLD_PATH_OTO_HERING_OUTLETS + 'products/', ['PRODUCT_ID'])
        update_new_data(spark, dzarm, GOLD_PATH_OTO_DZARM + 'products/', ['PRODUCT_ID'])
        
        # Remove as colunas desnecessárias
        
        # Envia somente os dados incrementais dos últimos 3 dias
        send_to_ftp(her_store, 'heringhomolog', 'products')
        send_to_ftp(her_kids, 'heringkids', 'products')
        send_to_ftp(her_outlet, 'heringoutlet', 'products')
        send_to_ftp(dzarm, 'dzarm', 'products')
    
    else:
        write_new_data(spark, her_store, GOLD_PATH_OTO_HERING_HMG + 'products/', 'delta')
        write_new_data(spark, her_kids, GOLD_PATH_OTO_HERING_KIDS + 'products/', 'delta')
        write_new_data(spark, her_outlet, GOLD_PATH_OTO_HERING_OUTLETS + 'products/', 'delta')
        write_new_data(spark, dzarm, GOLD_PATH_OTO_DZARM + 'products/', 'delta')